# Building a Local Agent with Strands Agents and Ollama Model

This notebook demonstrates how to create a personal agent using Strands Agent and Ollama. The agent will be capable of performing various local tasks such as file operations, web searches, and system commands.

## What is Ollama?

[Ollama](https://ollama.com/) is an open-source framework that allows you to run large language models (LLMs) locally on your machine. It provides a simple API for interacting with these models, making it ideal for privacy-focused applications where you don't want to send data to external services.

Key benefits of Ollama:
- **Privacy**: All processing happens locally on your machine
- **No API costs**: Free to use as much as you want
- **Offline capability**: Works without internet connection
- **Customization**: Can be fine-tuned for specific use 


## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |Ollama Model - to create a file operations agent   |
|Agent Structure     |Single agent architecture                          |


</div>


### Agent Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [1]:
pip install -r requirements.txt


   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   ---------------------------------------- 0/2 [PyPDF2]
   -------------------- ------------------- 1/2 [ollama]
   ---------------------------------------- 2/2 [ollama]

Note: you may need to restart the kernel to use updated packages.


## Setup and Installation

Before running this notebook, make sure you have:

1. Install Ollama: Follow instructions at [https://ollama.com/download](https://ollama.com/download)
2. Start the Ollama server: `ollama serve`
3. Downloaded a model with Ollama: `ollama pull llama3.2:1b`

Refer to the [documentation](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) for detailed instructions.

In this notebook, we will download Ollama for the linux distribution for compatibility with SageMaker Studio. This is done for code execution during AWS lead workshops on Workshop Studio. If you are running this code locally, you should adjust the steps to download Ollama to your current enviroment.

In [ ]:
import subprocess

OLLAMA_PATH = r"C:\Users\TEJAS\AppData\Local\Programs\Ollama\ollama.exe"  # <-- use correct path

# Test version
result = subprocess.run([OLLAMA_PATH, "--version"], capture_output=True, text=True)
print(result.stdout)

# Example query
def ask_ollama(prompt: str) -> str:
    result = subprocess.run([OLLAMA_PATH, "query", "llama2", prompt], capture_output=True, text=True)
    return result.stdout

print(ask_ollama("Summarize Python in 3 bullet points"))


ollama version is 0.13.4




In [11]:
import subprocess

OLLAMA_PATH = r"C:\Users\TEJAS\AppData\Local\Programs\Ollama\ollama.exe"

def ask_ollama(prompt: str) -> str:
    result = subprocess.run([OLLAMA_PATH, "query", "llama2", prompt], capture_output=True, text=True)
    return result.stdout


In [12]:
import subprocess
import os

# Full path to Ollama executable
OLLAMA_PATH = r"C:\Users\TEJAS\AppData\Local\Programs\Ollama\ollama.exe"

# Function to query Ollama
def ask_ollama(prompt: str) -> str:
    result = subprocess.run([OLLAMA_PATH, "query", "llama2", prompt],
                            capture_output=True, text=True)
    return result.stdout

# Local file operations
def list_files(directory="."):
    return "\n".join(os.listdir(directory))

def read_file(file_path):
    with open(file_path, "r") as f:
        return f.read()

def write_file(file_path, content):
    with open(file_path, "w") as f:
        f.write(content)
    return f"File '{file_path}' created."


In [13]:
files = list_files()
print("Files in current directory:\n", files)


Files in current directory:
 images
ollama_file_ops_agent.ipynb
requirements.txt
sample_file


In [14]:
print(write_file("sample.txt", "Hello! This is a test file created by Ollama agent."))


File 'sample.txt' created.


In [15]:
content = read_file("sample.txt")
print("File content:\n", content)

summary = ask_ollama(f"Summarize the following text in 3 bullet points:\n{content}")
print(summary)


File content:
 Hello! This is a test file created by Ollama agent.



In [16]:
from strands.models.ollama import OllamaModel

# Configure the Ollama model
ollama_model = OllamaModel(
    model_id="llama3.2:latest", # Make sure this is the model you downloaded from Ollama
    host="http://localhost:11434",
    params={
        "max_tokens": 2048,  # Adjust based on your model's capabilities
        "temperature": 0.7,  # Lower for more deterministic responses, higher for more creative
        "top_p": 0.9,        # Nucleus sampling parameter
        "stream": True,      # Enable streaming responses
    },
)

C:\Users\TEJAS\AppData\Roaming\Python\Python314\site-packages\IPython\core\interactiveshell.py:3701: UserWarning: Invalid configuration parameters: ['params'].
Valid parameters are: ['additional_args', 'keep_alive', 'max_tokens', 'model_id', 'options', 'stop_sequences', 'temperature', 'top_p'].

See https://github.com/strands-agents/sdk-python/issues/815
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# this will work on linux computers
curl -fsSL https://ollama.com/install.sh | sh

SyntaxError: invalid syntax (2800989438.py, line 2)

In [ ]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

In [ ]:
!ollama pull llama3.2:3b

In [ ]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [ ]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

## Defining Custom Tools

Tools are functions that the agent can use to interact with the environment. Below, we define several useful tools for our personal agent.

In [ ]:
# File Operation Tools
@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content. Supports both text and PDF files.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        # Check if it's a PDF file
        if file_path.lower().endswith('.pdf'):
            import PyPDF2
            with open(file_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text if text.strip() else "Error: Could not extract text from PDF"
        else:
            # Regular text file
            with open(file_path, "r", encoding="utf-8") as file:
                return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Creating the Ollama-powered Agent

Now we'll create our agent using the Ollama model and the tools we defined above.

Note: You can add more tools like `execute_commands`, `search_files` etc.

In [ ]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:3b"  # You can change this to any model you have pulled with Ollama.
)

#### Configure the Ollama model
Make sure your Ollama service is running at http://localhost:11434 and your `model_id` is in the list of Ollama models printed above.

In [ ]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    max_tokens=4096,  # Adjust based on your model's capabilities
    temperature=0.7,  # Lower for more deterministic responses, higher for more creative
    top_p=0.9,  # Nucleus sampling parameter
)

# Create the agent
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## Testing the Agent

Let's test our agent with some example tasks.

In [ ]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)

In [ ]:
# Example 2: List files in the current directory
response = local_agent("Show me the files in the current directory")

In [ ]:
# Example 3: Create a sample file
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'"
)

In [ ]:
# Example 4: create a readme file after reading and understanding multiple files
response = local_agent("Create a readme.md for the current directory")

## Conclusion

In this notebook, we've created a local personal agent using Stands and Ollama. The agent can perform file operations (read, write, append) and Summarize/Analyze text

This demonstrates the power of running AI models locally with Ollama, combined with the flexibility of strands' tool system. You can extend this agent by adding more tools or using different Ollama models based on your needs.

### Next Steps (Ideas)

- Try different Ollama models to see how they affect the agent's capabilities
- Add more complex tools like web search, email sending, or calendar integration
- Implement memory for the agent to remember past interactions
- Create a simple UI for interacting with your agent